In [1]:
import os
import math
import numpy as np
import pandas as pd
import argparse

from skfda import FDataGrid
from skfda.representation.basis import BSpline
from skfda.preprocessing.smoothing import BasisSmoother
from skfda.preprocessing.registration import ElasticRegistration, landmark_registration

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from skfda.exploratory.depth import IntegratedDepth, ModifiedBandDepth

from cycler import cycler
from matplotlib import pyplot as plt

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.metrics import (
	precision_score, recall_score, f1_score, balanced_accuracy_score, roc_curve, confusion_matrix
)


agg_columns = ['patient_id', 'slice_id', 'img_type']

n_basis=18
order=4

prc_rm=0.05
n_points=111

basis = BSpline(domain_range=(0, 1), n_basis=n_basis, order=order)
smoother = BasisSmoother(basis=basis, return_basis=True, method='svd')

registration = ElasticRegistration()
ID = IntegratedDepth()
MBD = ModifiedBandDepth()


def get_model():
	return {
		'svm': SVC(class_weight='balanced', probability=True),
		'rf': RandomForestClassifier(class_weight='balanced', n_jobs=4),
		'xgb': xgb.XGBClassifier(tree_method='gpu_hist', eval_metric='logloss', use_label_encoder=False),
	}[MODEL]


def cut_ends(bsplined, order=0, prc_rm_start=prc_rm, prc_rm_end=prc_rm, n_points=n_points):
	bsplined_grid = bsplined.derivative(order=order).to_grid(np.linspace(0, 1, n_points))
	return FDataGrid(
		data_matrix=bsplined_grid.data_matrix[
			..., int(n_points * prc_rm_start): int(n_points * (1 - prc_rm_end)), 0
		],
		grid_points=bsplined_grid.grid_points[0][
			int(n_points * prc_rm_start): int(n_points * (1 - prc_rm_end))
		]
	)


def get_landmark_registration(bsplined, order=0):
	bsplined_grid = cut_ends(bsplined, order)
	landmark_indexes = cut_ends(bsplined, order, prc_rm_end=0.5).data_matrix.argmax(axis=1)
	grid_points = bsplined_grid.grid_points[0]
	landmarks = [grid_points[index] for index in np.concatenate(landmark_indexes)]
	return landmark_registration(bsplined_grid, landmarks)
		


## Print curves of each patient

In [ ]:
segmentation_id = 4
# patients = [3, 6, 7, 29, 66, 76, 108, 119, 133, 140]
patients = [2, 15, 28, 32, 35, 39, 40, 41, 45, 50, 52, 64, 66]
bspline_levels = ['patient_id', 'slice_id']
min_n_curves = 5

dataset = (
	pd.read_csv(
		f'./segmentations/segmentation-{segmentation_id}.csv',
		dtype={
			'img_type': int,
			'patient_id': int,
			'cycle_id': int,
			'slice_id': int,
			'label': bool,
			'mask_int_mean': float,
			'segment': int,
		},
	)
		.drop_duplicates()
		.sort_values(agg_columns + ['cycle_id'])
)
dataset = dataset[dataset.patient_id.apply(lambda x: x in patients)]
dataset = dataset.merge(dataset.query('label')[bspline_levels].drop_duplicates())
ts = (
	dataset[bspline_levels + ['cycle_id']].drop_duplicates()
		.groupby(bspline_levels).cycle_id.count()
		.apply(lambda x: np.linspace(0, 1, int(x)))
		.reset_index()
)

dataset = dataset.groupby(agg_columns + ['label']).mask_int_mean.apply(list).reset_index()
labels = dataset.groupby(bspline_levels).label.apply(list).reset_index()
dataset = dataset.groupby(bspline_levels).mask_int_mean.apply(list).reset_index().merge(ts)
dataset = dataset[dataset.mask_int_mean.apply(lambda x: len(x) >= min_n_curves)]
dataset['bsplined'] = dataset.apply(
	lambda x: smoother.fit_transform(
		FDataGrid(data_matrix=x['mask_int_mean'], grid_points=x['cycle_id'])
	),
	axis='columns',
)
# dataset['bsplined'] = dataset.bsplined.apply(get_landmark_registration, order=1)
dataset['bsplined'] = dataset.bsplined.apply(cut_ends, order=1)
dataset = dataset[bspline_levels + ['bsplined']]
dataset = dataset.merge(labels)


In [ ]:
def print_patient(row, segmentation_id):
	colors = ['red' if x else 'green' for x in row['label']]
	alphas = [1 if x else 0.1 for x in row['label']]
	color_cycle = cycler(color=colors, alpha=alphas)

	fig = plt.figure(figsize=(11, 7))
	ax = plt.axes(xlabel='t')
	ax.set_prop_cycle(color_cycle)
	row['bsplined'].plot(fig=fig, ax=ax)
	name = f'./patient_plots/patient-{row["patient_id"]}-segment-{segmentation_id}.png'
	if 'slice_id' in dataset.columns:
		name = f'./patient_slice_plots_unregistered/patient-{row["patient_id"]}-slice{row["slice_id"]}-segment-{segmentation_id}.png'
	plt.savefig(name, dpi=150)
	plt.close()


x = dataset.apply(print_patient, segmentation_id=segmentation_id, axis='columns')

## Generate dataset

In [5]:
FILE = 'features.csv'

In [ ]:
train_n_points = 100


def get_descrete_points(fd_smooth):
	t_cut = np.linspace(
		start=0,
		stop=fd_smooth.data_matrix.shape[1] - 1,
		num=train_n_points, endpoint=True, dtype=int,
	)
	return fd_smooth.data_matrix[:, t_cut, 0]


def cart2pol(x, y):
	rho = np.sqrt(x**2 + y**2)
	phi = np.arctan2(y, x)
	return(rho, phi)


segmentations = []

for segmentation_id in range(0, 25):
	print(segmentation_id)
	file = f'./segmentations/segmentation-{segmentation_id}.csv'
	dataset = (
	pd.read_csv(
		file,
		dtype={
			'img_type': int,
			'patient_id': int,
			'cycle_id': int,
			'slice_id': int,
			'label': bool,
			'mask_int_mean': float,
			'segment': int,
		},
	)
	.drop_duplicates()
	.sort_values(agg_columns + ['cycle_id'])
	)
	dataset = dataset.merge(dataset.query('label').patient_id.drop_duplicates())
	# dataset = dataset[dataset.patient_id.apply(lambda x: x not in [3, 6, 29, 86, 108, 119, 140])]
	dataset = dataset[dataset.patient_id.apply(lambda x: x in [2, 15, 28, 32, 35, 39, 40, 41, 45, 50, 52, 64, 66])]
	ts = (
		dataset[['patient_id', 'cycle_id']].drop_duplicates()
			.groupby('patient_id').cycle_id.count()
			.apply(lambda x: np.linspace(0, 1, int(x)))
			.reset_index()
	)

	dataset = dataset.groupby(agg_columns + ['label']).mask_int_mean.apply(list).reset_index()
	bsplined = dataset.groupby('patient_id').mask_int_mean.apply(list).reset_index().merge(ts)
	bsplined = bsplined.apply(
		lambda x: smoother.fit_transform(
			FDataGrid(data_matrix=x['mask_int_mean'], grid_points=x['cycle_id'])
		),
		axis='columns',
	)
	print('extracting features ...')
	registered = [get_landmark_registration(fd_smooth, 1) for fd_smooth in bsplined]
	unregistered = [cut_ends(fd_smooth, 1) for fd_smooth in bsplined]
	
	dataset['unregistered_ids'] = np.concatenate(
		[ID(fd_smooth).reshape(-1, 1) for fd_smooth in unregistered]
	)
	dataset['unregistered_mbds'] = np.concatenate(
		[MBD(fd_smooth).reshape(-1, 1) for fd_smooth in unregistered]
	)
	dataset['unregistered_max_values'] = np.concatenate(
		[fd_smooth.data_matrix.max(axis=1).reshape(-1, 1) for fd_smooth in unregistered]
	)
	
	dataset['registered_ids'] = np.concatenate(
		[ID(fd_smooth).reshape(-1, 1) for fd_smooth in registered]
	)
	dataset['registered_mbds'] = np.concatenate(
		[MBD(fd_smooth).reshape(-1, 1) for fd_smooth in registered]
	)
	dataset['registered_max_values'] = np.concatenate(
		[fd_smooth.data_matrix.max(axis=1).reshape(-1, 1) for fd_smooth in registered]
	)
	registered = [get_descrete_points(fd_smooth) for fd_smooth in registered]
	unregistered = [get_descrete_points(fd_smooth) for fd_smooth in unregistered]

	segmentations += [
		dataset.drop('mask_int_mean', axis='columns')
			.assign(segmentation_id=segmentation_id)
			.join(
				pd.DataFrame(
					np.concatenate(registered),
					columns=[f'registered_discrete_{i}' for i in range(100)],
				)
			)
			.join(
				pd.DataFrame(
					np.concatenate(unregistered),
					columns=[f'unregistered_discrete_{i}' for i in range(100)],
				)
			)
	]

dataset = pd.concat(segmentations)
features = 'registered_discrete'
registered_polar = [
	[cart2pol(0.01, x - y) for x, y in zip(row, np.concatenate([[0], row[:-1]]))]
	for row in dataset[[f'{features}_{i}' for i in range(100)]].to_numpy()
]
registered_polar = (
	pd.DataFrame(
		[np.concatenate(row) for row in registered_polar],
		columns=np.concatenate(
			[
				[f'registered_polar_{i}_rho', f'registered_polar_{i}_phi']
				for i in range(100)
			]
		)
	)
)

features = 'unregistered_discrete'
unregistered_polar = [
	[cart2pol(0.01, x - y) for x, y in zip(row, np.concatenate([[0], row[:-1]]))]
	for row in dataset[[f'{features}_{i}' for i in range(100)]].to_numpy()
]
unregistered_polar = (
	pd.DataFrame(
		[np.concatenate(row) for row in unregistered_polar],
		columns=np.concatenate(
			[
				[f'unregistered_polar_{i}_rho', f'unregistered_polar_{i}_phi']
				for i in range(100)
			]
		)
	)
)
dataset.join(registered_polar).join(unregistered_polar).to_csv(FILE, index=False)

## Learn on patients from generated features

#### With CV

In [ ]:
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import StratifiedKFold


def specificity(y_true, y_pred, zero_division=0):
	tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
	if tn+fp == 0 and zero_division:
		return zero_division
	return tn / (tn+fp)


def train_eval(X, Y, model_producer, metrics=[f1_score]):
	metric_results = [[] for _ in metrics]
	for indices in k_folds.split(X, Y):
		train_indices, test_indices = indices
		X_train, X_test = X[train_indices], X[test_indices]
		y_train, y_test = Y[train_indices], Y[test_indices]
		neg_class_weight = (y_train == 1).sum() / len(y_train)
		weights = [
			neg_class_weight if label == 0 else 1 - neg_class_weight
			for label in y_train
		]
		scaler = MinMaxScaler(feature_range=(0, 1), copy=False)
		scaler = scaler.fit(X_train)
		X_train = scaler.transform(X_train)
		X_test = scaler.transform(X_test)
		model = model_producer()
		model.fit(
			X_train, y_train,
			sample_weight=weights,
			eval_set=[( X_train, y_train), ( X_test, y_test)],
			eval_metric="auc",
			verbose=False,
		)

		pred = model.predict(X_test)
		for metric_result, metric in zip(metric_results, metrics):
			metric_result.append(metric(y_test, pred))
	return metric_results


os.environ['HYPEROPT_FMIN_SEED'] = "1"
k_folds = StratifiedKFold(n_splits=5)

space = {
	'max_depth': hp.quniform("max_depth", 3, 18, 1),
	'gamma': hp.uniform('gamma', 0, 1),
	'reg_alpha' : hp.uniform('reg_alpha', 0, 1),
	'colsample_bytree' : hp.uniform('colsample_bytree', .4, .8),
	'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
	'n_estimators': 180,
	'seed': 0
}

result = []
dataset = pd.read_csv(FILE)
features = 'unregistered_discrete'

for patient, segmentation in (
	dataset[['patient_id', 'segmentation_id']]
		.drop_duplicates().to_records(index=False)
):
	print(f'patient - {patient}, segmentation - {segmentation}')
	dataset_group = dataset.query(
		f'segmentation_id == {segmentation} and patient_id == {patient}'
	)
	y = dataset_group.label.astype(int).to_numpy()
	X = dataset_group[[f'{features}_{i}' for i in range(100)]].to_numpy()

	def objective(space):
		create_model = lambda: xgb.XGBClassifier(
			use_label_encoder=False,
			eval_metric='auc',
			n_estimators=space['n_estimators'],
			max_depth=int(space['max_depth']),
			gamma=space['gamma'],
			reg_alpha=space['reg_alpha'],
			min_child_weight=space['min_child_weight'],
			colsample_bytree=space['colsample_bytree'],
		)
		f1 = np.mean(train_eval(X, y, create_model)[0])
		return {'loss': -f1, 'status': STATUS_OK }


	trials = Trials()

	best_hyperparams = fmin(
		fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials
	)

	create_model = lambda: xgb.XGBClassifier(
		use_label_encoder=False,
		eval_metric='auc',
		n_estimators=space['n_estimators'],
		seed=space['seed'],
		max_depth=int(best_hyperparams['max_depth']),
		gamma=best_hyperparams['gamma'],
		reg_alpha=best_hyperparams['reg_alpha'],
		min_child_weight=best_hyperparams['min_child_weight'],
		colsample_bytree=best_hyperparams['colsample_bytree'],
	)
	metrics = train_eval(
		X, y, create_model,
		[precision_score, recall_score, f1_score, balanced_accuracy_score, specificity]
	)

	metrics = {
		'patient_id': patient,
		'segmentation_id': segmentation,
		'precision_mean': np.mean(metrics[0]),
		'precision_std': np.std(metrics[0]),
		'recall_mean': np.mean(metrics[1]),
		'recall_std': np.std(metrics[1]),
		'f1_mean':np.mean( metrics[2]),
		'f1_std':np.std( metrics[2]),
		'balanced_accuracy_mean': np.mean(metrics[3]),
		'balanced_accuracy_std': np.std(metrics[3]),
		'specificity_mean': np.mean(metrics[4]),
		'specificity_std': np.std(metrics[4]),
	}
	result += [{**metrics, **best_hyperparams}]


pd.DataFrame(result).to_csv(f'./show/{features}.csv', index=False)

#### Without CV XGBoost

In [ ]:
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split
import re

def specificity(y_true, y_pred, zero_division=0):
	tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
	if tn+fp == 0 and zero_division:
		return zero_division
	return tn / (tn+fp)


os.environ['HYPEROPT_FMIN_SEED'] = "1"

space = {
	'max_depth': hp.quniform("max_depth", 3, 18, 1),
	'gamma': hp.uniform('gamma', 0, 1),
	'reg_alpha' : hp.uniform('reg_alpha', 0, 1),
	'colsample_bytree' : hp.uniform('colsample_bytree', .4, .8),
	'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
	'n_estimators': 180,
	'seed': 0
}

result = []
dataset = pd.read_csv(FILE)
features = r'^unregistered_(discrete_\d+|ids|mbds|max_values)'
result_file = 'unregistered_all'

for patient, segmentation in (
	dataset[['patient_id', 'segmentation_id']]
		.query('segmentation_id == 5')
		.drop_duplicates().to_records(index=False)
):
	print(f'patient - {patient}, segmentation - {segmentation}')
	dataset_group = dataset.query(
		f'segmentation_id == {segmentation} and patient_id == {patient}'
	)
	y = dataset_group.label.astype(int).to_numpy()
	X = dataset_group[[name for name in dataset.columns if re.match(features, name)]].to_numpy()
	if patient == 2:
		print(X.shape)

	X_train, X_test, y_train, y_test = train_test_split(
		X, y, test_size = 0.3, random_state = 0, stratify=y
	)
	
	neg_class_weight = (y_train == 1).sum() / len(y_train)
	weights = [
		neg_class_weight if label == 0 else 1 - neg_class_weight
		for label in y_train
	]
	scaler = MinMaxScaler(feature_range=(0, 1), copy=False)
	scaler = scaler.fit(X_train)
	X_train = scaler.transform(X_train)
	X_test = scaler.transform(X_test)


	def objective(space):
		model = xgb.XGBClassifier(
			use_label_encoder=False,
			eval_metric='auc',
			n_estimators=space['n_estimators'],
			max_depth=int(space['max_depth']),
			gamma=space['gamma'],
			reg_alpha=space['reg_alpha'],
			min_child_weight=space['min_child_weight'],
			colsample_bytree=space['colsample_bytree'],
		)
		
		model.fit(
			X_train, y_train,
			sample_weight=weights,
			eval_set=[( X_train, y_train), ( X_test, y_test)],
			eval_metric="auc",
			verbose=False,
		)

		pred = model.predict(X_test)
		return {'loss': -f1_score(y_test, pred), 'status': STATUS_OK }


	trials = Trials()

	best_hyperparams = fmin(
		fn=objective, space=space, algo=tpe.suggest, max_evals=400, trials=trials
	)

	model = xgb.XGBClassifier(
		use_label_encoder=False,
		eval_metric='auc',
		n_estimators=space['n_estimators'],
		max_depth=int(best_hyperparams['max_depth']),
		gamma=best_hyperparams['gamma'],
		reg_alpha=best_hyperparams['reg_alpha'],
		min_child_weight=best_hyperparams['min_child_weight'],
		colsample_bytree=best_hyperparams['colsample_bytree'],
	)

	model.fit(
		X_train, y_train,
		sample_weight=weights,
		eval_set=[( X_train, y_train), ( X_test, y_test)],
		eval_metric="auc",
		verbose=False,
	)

	pred = model.predict(X_test)
	metrics = {
		'patient_id': patient,
		'segmentation_id': segmentation,
		'precision': precision_score(y_test, pred),
		'recall': recall_score(y_test, pred),
		'f1': f1_score(y_test, pred),
		'balanced_accuracy': balanced_accuracy_score(y_test, pred),
		'specificity': specificity(y_test, pred),
	}
	result += [{**metrics, **best_hyperparams}]


pd.DataFrame(result).to_csv(f'./show/{result_file}_no_cv.csv', index=False)

#### Without CV NN

In [ ]:
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split
import re

def specificity(y_true, y_pred, zero_division=0):
	tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
	if tn+fp == 0 and zero_division:
		return zero_division
	return tn / (tn+fp)


os.environ['HYPEROPT_FMIN_SEED'] = "1"

space = {
	'max_depth': hp.quniform("max_depth", 3, 18, 1),
	'gamma': hp.uniform('gamma', 0, 1),
	'reg_alpha' : hp.uniform('reg_alpha', 0, 1),
	'colsample_bytree' : hp.uniform('colsample_bytree', .4, .8),
	'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
	'n_estimators': 180,
	'seed': 0
}

result = []
dataset = pd.read_csv(FILE)
features = r'^unregistered_(discrete_\d+|ids|mbds|max_values)'
result_file = 'unregistered_all'

for patient, segmentation in (
	dataset[['patient_id', 'segmentation_id']]
		.query('segmentation_id == 5')
		.drop_duplicates().to_records(index=False)
):
	print(f'patient - {patient}, segmentation - {segmentation}')
	dataset_group = dataset.query(
		f'segmentation_id == {segmentation} and patient_id == {patient}'
	)
	y = dataset_group.label.astype(int).to_numpy()
	X = dataset_group[
		[name for name in dataset.columns if re.match(features, name)]
	].to_numpy()
	if patient == 2:
		print(X.shape)

	X_train, X_test, y_train, y_test = train_test_split(
		X, y, test_size = 0.3, random_state = 0, stratify=y
	)
	
	neg_class_weight = (y_train == 1).sum() / len(y_train)
	weights = [
		neg_class_weight if label == 0 else 1 - neg_class_weight
		for label in y_train
	]
	scaler = MinMaxScaler(feature_range=(0, 1), copy=False)
	scaler = scaler.fit(X_train)
	X_train = scaler.transform(X_train)
	X_test = scaler.transform(X_test)


	def objective(space):
		model = xgb.XGBClassifier(
			use_label_encoder=False,
			eval_metric='auc',
			n_estimators=space['n_estimators'],
			max_depth=int(space['max_depth']),
			gamma=space['gamma'],
			reg_alpha=space['reg_alpha'],
			min_child_weight=space['min_child_weight'],
			colsample_bytree=space['colsample_bytree'],
		)
		
		model.fit(
			X_train, y_train,
			sample_weight=weights,
			eval_set=[( X_train, y_train), ( X_test, y_test)],
			eval_metric="auc",
			verbose=False,
		)

		pred = model.predict(X_test)
		return {'loss': -f1_score(y_test, pred), 'status': STATUS_OK }


	trials = Trials()

	best_hyperparams = fmin(
		fn=objective, space=space, algo=tpe.suggest, max_evals=400, trials=trials
	)

	model = xgb.XGBClassifier(
		use_label_encoder=False,
		eval_metric='auc',
		n_estimators=space['n_estimators'],
		max_depth=int(best_hyperparams['max_depth']),
		gamma=best_hyperparams['gamma'],
		reg_alpha=best_hyperparams['reg_alpha'],
		min_child_weight=best_hyperparams['min_child_weight'],
		colsample_bytree=best_hyperparams['colsample_bytree'],
	)

	model.fit(
		X_train, y_train,
		sample_weight=weights,
		eval_set=[( X_train, y_train), ( X_test, y_test)],
		eval_metric="auc",
		verbose=False,
	)

	pred = model.predict(X_test)
	metrics = {
		'patient_id': patient,
		'segmentation_id': segmentation,
		'precision': precision_score(y_test, pred),
		'recall': recall_score(y_test, pred),
		'f1': f1_score(y_test, pred),
		'balanced_accuracy': balanced_accuracy_score(y_test, pred),
		'specificity': specificity(y_test, pred),
	}
	result += [{**metrics, **best_hyperparams}]


pd.DataFrame(result).to_csv(f'./show/{result_file}_no_cv.csv', index=False)

## Learn on each patient

In [2]:
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import seaborn as sns
from tqdm import tqdm
import cmath

from sklearn.model_selection import GridSearchCV


scaler = MinMaxScaler(feature_range=(0, 1), copy=False)
param_grid = {
	# 'learning_rate': np.linspace(0.000001, 1, 15),
	# 'n_estimators ': list(range(1, 5))
	'reg_alpha': np.linspace(0.000001, 0.1, 15),
	# 'reg_lambda': np.linspace(0.000001, 0.1, 15),
}

k_folds = StratifiedKFold(n_splits=5)
train_n_points = 100
FOLDER = 'roc_unregistered'
FILE = 'reg_alpha_v1'


def get_descrete_points(fd_smooth):
	t_cut = np.linspace(
		start=0,
		stop=fd_smooth.data_matrix.shape[1] - 1,
		num=train_n_points, endpoint=True, dtype=int,
	)
	return fd_smooth.data_matrix[:, t_cut, 0]


def get_polar_points(fd_smooth):
	coefs = np.fft.fft(
		fd_smooth.data_matrix
			.reshape(fd_smooth.data_matrix.shape[0], fd_smooth.data_matrix.shape[1])
	)
	return (
		np.array([[cmath.polar(coef) for coef in row] for row in coefs])
			.reshape(coefs.shape[0], -1)
	)


def grid_search(Y, X, segmentation_id, patient_id):
	model = xgb.XGBClassifier(
		tree_method='gpu_hist', eval_metric='logloss', use_label_encoder=False
	)
	search_result = GridSearchCV(model, param_grid, scoring='f1').fit(X, Y.ravel())
	gs = pd.DataFrame(search_result.cv_results_['params'])
	gs['segmentation_id'] = segmentation_id
	gs['patient_id'] = patient_id
	gs['mean_test_score'] = search_result.cv_results_[f'mean_test_score']
	gs['std_test_score'] = search_result.cv_results_[f'std_test_score']
	gs = gs.iloc[[search_result.best_index_]]
	if not os.path.exists(f'./{FOLDER}/{FILE}.csv'):
		gs.to_csv(f'./{FOLDER}/{FILE}.csv', index=False)
	else:
		gs.to_csv(f'./{FOLDER}/{FILE}.csv', index=False, mode='a', header=False)
	return search_result.best_params_


def train_print_rocs(Y, X, segmentation_id, patient_id, reg_alpha):
	# ax = None
	# best_params = grid_search(Y, X, segmentation_id, patient_id)

	for i, indices in enumerate(k_folds.split(X, Y)):
		train_index, test_index = indices
		X_train, X_test = X[train_index], X[test_index]
		y_train, y_test = Y[train_index], Y[test_index]
		scaler = MinMaxScaler(feature_range=(0, 1), copy=False)
		scaler = scaler.fit(X_train)
		X_train = scaler.transform(X_train)
		X_test = scaler.transform(X_test)

		model = xgb.XGBClassifier(
			tree_method='gpu_hist', eval_metric='logloss', use_label_encoder=False,
			reg_alpha=reg_alpha,
			# learning_rate=best_params['learning_rate'],
			# reg_alpha=best_params['reg_alpha'],
			# reg_lambda=best_params['reg_lambda'],
		)
		model.fit(X_train, y_train)
		preds = model.predict_proba(X_test)[:,1]

		fpr, tpr, thresholds = roc_curve(y_test, preds)
		roc_scores = pd.DataFrame(
			{
				'fpr': fpr,
				'tpr': tpr,
				'threshold': thresholds,
				'fold': i,
				'segmentation_id': segmentation_id,
				'patient_id': patient_id,
			}
		)
		if not os.path.exists(f'./show/{ROC_FILE}.csv'):
			roc_scores.to_csv(f'./show/{ROC_FILE}.csv', index=False)
		else:
			roc_scores.to_csv(f'./show/{ROC_FILE}.csv', index=False, mode='a', header=False)
		ax = sns.lineplot(x=fpr, y=tpr, ax=ax)

	ax.set_ylabel('False positive rate')
	ax.set_xlabel('True positive rate')
	plt.savefig(f'./roc_unregistered/xgb_segmentation-{segmentation_id}-patient-{patient_id}.png')
	plt.close()


def process_patient(patient, segmentation_id):
	ids = ID(patient['fd_smooth']).reshape(-1, 1)
	mbds = MBD(patient['fd_smooth']).reshape(-1, 1)
	max_values = patient['fd_smooth'].data_matrix.max(axis=1).reshape(-1, 1)
	# descret_points = get_descrete_points(patient['fd_smooth'])
	polar_points = get_polar_points(patient['fd_smooth'])

	train_set = np.hstack([ids, mbds, max_values, polar_points])
	labels = np.asarray(patient['label']).astype(int).reshape(-1, 1)
	# grid_search(labels, train_set, segmentation_id, patient['patient_id'])
	train_print_rocs(labels, train_set, segmentation_id, patient['patient_id'])


In [13]:

def train_roc(Y, X, segmentation_id, patient_id, param):
	pd_roc_scores = []

	for i, indices in enumerate(k_folds.split(X, Y)):
		train_index, test_index = indices
		X_train, X_test = X[train_index], X[test_index]
		y_train, y_test = Y[train_index], Y[test_index]
		scaler = MinMaxScaler(feature_range=(0, 1), copy=False)
		scaler = scaler.fit(X_train)
		X_train = scaler.transform(X_train)
		X_test = scaler.transform(X_test)

		model = xgb.XGBClassifier(
			tree_method='gpu_hist', eval_metric='logloss', use_label_encoder=False,
			reg_alpha=param,
			# learning_rate=param,
		)
		model.fit(X_train, y_train)
		preds = model.predict_proba(X_test)[:,1]

		fpr, tpr, thresholds = roc_curve(y_test, preds)
		gmean = np.sqrt(tpr * (1 - fpr))
		index = np.argmax(gmean)
		roc_scores = pd.DataFrame(
			{
				'fold': [i],
				'gmean': gmean[index],
				'fpr': fpr[index],
				'tpr': tpr[index],
				'threshold': thresholds[index],
				'segmentation_id': segmentation_id,
				'patient_id': patient_id,
				'param': param,
				'f1': f1_score(y_test, preds > thresholds[index])
			}
		)
		pd_roc_scores += [roc_scores]

	return pd_roc_scores


def process_patient(patient, segmentation_id):
	ids = ID(patient['fd_smooth']).reshape(-1, 1)
	mbds = MBD(patient['fd_smooth']).reshape(-1, 1)
	max_values = patient['fd_smooth'].data_matrix.max(axis=1).reshape(-1, 1)
	# points = get_polar_points(patient['fd_smooth'])
	points = get_descrete_points(patient['fd_smooth'])

	train_set = np.hstack([ids, mbds, max_values, points])
	labels = np.asarray(patient['label']).astype(int).reshape(-1, 1)
	pd_roc_scores = []
	
	for param in np.linspace(0.000001, 0.1, 15): # reg_alpha
	# for param in np.linspace(0.000001, 1, 15): # lr
		pd_roc_scores += train_roc(
			labels, train_set, segmentation_id, patient['patient_id'], param
		)
	pd_roc_scores = pd.concat(pd_roc_scores)
	pd_roc_scores = (
		pd_roc_scores.groupby('fold').f1.max().reset_index().merge(pd_roc_scores)
			.groupby('fold').gmean.max().reset_index().merge(pd_roc_scores)
			.groupby('fold').threshold.min().reset_index().merge(pd_roc_scores)
	)
	if not os.path.exists(f'./show/{ROC_FILE}.csv'):
		pd_roc_scores.to_csv(f'./show/{ROC_FILE}.csv', index=False)
	else:
		pd_roc_scores.to_csv(f'./show/{ROC_FILE}.csv', index=False, mode='a', header=False)


In [ ]:
tqdm.pandas()

# ROC_FILE = 'registered_polar_reg_alpha'
# ROC_FILE = 'registered_polar_lr'
# ROC_FILE = 'registered_discrete_reg_alpha'
ROC_FILE = 'registered_discrete_lr'


for segmentation_id in range(0, 25):
	print(segmentation_id)
	file = f'./segmentations/segmentation-{segmentation_id}.csv'
	dataset = (
	pd.read_csv(
		file,
		dtype={
			'img_type': int,
			'patient_id': int,
			'cycle_id': int,
			'slice_id': int,
			'label': bool,
			'mask_int_mean': float,
			'segment': int,
		},
	)
	.drop_duplicates()
	.sort_values(agg_columns + ['cycle_id'])
	)
	dataset = dataset.merge(dataset.query('label').patient_id.drop_duplicates())
	# dataset = dataset[dataset.patient_id.apply(lambda x: x not in [3, 6, 29, 86, 108, 119, 140])]
	dataset = dataset[dataset.patient_id.apply(lambda x: x in [2, 15, 28, 32, 35, 39, 40, 41, 45, 50, 52, 64, 66])]
	ts = (
		dataset[['patient_id', 'cycle_id']].drop_duplicates()
			.groupby('patient_id').cycle_id.count()
			.apply(lambda x: np.linspace(0, 1, int(x)))
			.reset_index()
	)

	dataset = dataset.groupby(agg_columns + ['label']).mask_int_mean.apply(list).reset_index()
	bsplined = dataset.groupby('patient_id').mask_int_mean.apply(list).reset_index().merge(ts)
	bsplined = bsplined.apply(
		lambda x: smoother.fit_transform(
			FDataGrid(data_matrix=x['mask_int_mean'], grid_points=x['cycle_id'])
		),
		axis='columns',
	)
	print('registering ...')
	bsplined = [get_landmark_registration(fd_smooth, 1) for fd_smooth in bsplined]
	# bsplined = [cut_ends(fd_smooth, 1) for fd_smooth in bsplined]

	dataset = dataset.groupby('patient_id').label.apply(list).reset_index()
	dataset['fd_smooth'] = bsplined
	process_patient(dataset.iloc[0], segmentation_id)
	dataset.progress_apply(
		process_patient,
		segmentation_id=segmentation_id,
		axis='columns',
	)


In [ ]:
import seaborn as sns

%matplotlib ipympl

plt.style.use('dark_background')


In [ ]:

plt.close()

metric = 'learning_rate'

grid_search = pd.read_csv(f'./roc_unregistered/{metric}.csv')
sns.scatterplot(
	data=grid_search
		.groupby('patient_id')
		.mean_test_score.max()
		.reset_index()
		.merge(grid_search),
	y='segmentation_id', x=metric, hue='patient_id',
	palette=sns.color_palette('tab20')[:13],
)
plt.savefig(f'./roc_unregistered/{metric}-vs-segmentation.png')
plt.close()

sns.lineplot(
	data=grid_search
		.groupby(['patient_id', 'segmentation_id'])
		.mean_test_score.max()
		.reset_index()
		.merge(grid_search),
	y='mean_test_score', x='segmentation_id', hue='patient_id',
	palette=sns.color_palette('tab20')[:13],
)
plt.savefig(f'./roc_unregistered/{metric}-segmentation.png')
plt.close()

sns.lineplot(
	data=grid_search
		.groupby(['patient_id', metric])
		.mean_test_score.max()
		.reset_index()
		.merge(grid_search),
	y='mean_test_score', x=metric, hue='patient_id',
	palette=sns.color_palette('tab20')[:13],
)
plt.savefig(f'./roc_unregistered/{metric}.png')
plt.close()

### T-tests

In [ ]:
from scipy import stats

metric = 'reg_alpha'
# metric = 'learning_rate'
# folder = 'roc_unregistered'
folder = 'roc'
suffix = '-paired-registered'
# suffix = '-indep'


def ttest(sample):
	results = stats.ttest_rel(sample['x'], sample['y'])
	return (results.pvalue, results.statistic)


grid_search = pd.read_csv(f'./{folder}/{metric}_v0.csv')
# grid_search = pd.read_csv(f'./{folder}/{metric}.csv')

transformed = (
	grid_search
		.sort_values(['segmentation_id', 'patient_id'])
		.groupby('segmentation_id').mean_test_score.apply(list).reset_index()
		.assign(join_col=1)
)

matrix = (
	transformed.rename(
		{'segmentation_id': 'segmentation_x', 'mean_test_score': 'x'}, axis='columns',
	)
		.merge(
			transformed.rename(
				{'segmentation_id': 'segmentation_y', 'mean_test_score': 'y'},
				axis='columns',
			)
		)
)

matrix['statistic'] = matrix.apply(ttest, axis='columns')
matrix['pvalue'] = [x[0] for x in matrix.statistic]
matrix['statistic'] = [x[1] for x in matrix.statistic]
matrix = matrix[['segmentation_x', 'segmentation_y', 'statistic', 'pvalue']]
(
	matrix.pivot(index='segmentation_y', columns='segmentation_x', values='pvalue')
		.to_csv(f'./show/{metric}-pvalue{suffix}.csv')
)
(
	matrix.pivot(index='segmentation_y', columns='segmentation_x', values='statistic')
		.to_csv(f'./show/{metric}-statistic{suffix}.csv')
)

In [49]:
from scipy import stats

metric = 'reg_alpha'
# metric = 'learning_rate'
# folder = 'roc_unregistered'
folder = 'roc'
suffix = '-paired-registered'
# suffix = '-indep'


def friedman(data_file):
	transformed = (
		pd.read_csv(data_file)
		# pd.read_csv(f'./{folder}/{metric}.csv')
			.sort_values(['segmentation_id', 'patient_id'])
			.groupby('segmentation_id').mean_test_score.apply(list).tolist()
	)

	return stats.friedmanchisquare(*transformed)

result = [
	(optimized, registered, with_discrete, friedman(data_file))
	for data_file, optimized, registered, with_discrete in [
		('./roc/learning_rate_v1.csv', 'learning_rate', True, True),
		('./roc/learning_rate_both.csv', 'learning_rate', True, False),
		('./roc/reg_alpha_v1.csv', 'reg_alpha', True, True),
		('./roc/reg_alpha_both.csv', 'reg_alpha', True, False),
		
		('./roc_unregistered/learning_rate_v1.csv', 'learning_rate', False, True),
		('./roc_unregistered/learning_rate_both.csv', 'learning_rate', False, False),
		('./roc_unregistered/reg_alpha_v1.csv', 'reg_alpha', False, True),
		('./roc_unregistered/reg_alpha_both.csv', 'reg_alpha', False, False),
	]
]
(
	pd.DataFrame(result, columns=['optimized', 'registered', 'with_discrete', 'statistic'])
		.assign(
			pvalue=lambda y: [x[1] for x in y.statistic],
			statistic=lambda y: [x[0] for x in y.statistic],
		)
		.to_csv('./show/friednman.csv', index=False)
)

In [47]:
from scipy import stats

# metric = 'learning_rate'
metric = 'reg_alpha'
# version = 'both'
version = 'v1'


def wilcoxon_score(x):
	if x['registered'] == x['unregistered']:
		return None
	return stats.wilcoxon(x['registered'], x['unregistered'])



t_test = (
	pd.read_csv(f'./roc/{metric}_{version}.csv')
		.sort_values(['segmentation_id', 'patient_id'])
		.groupby('patient_id')
		.mean_test_score.apply(list).reset_index()
		.rename({'mean_test_score': 'registered'}, axis='columns')
		.merge(
			pd.read_csv(f'./roc_unregistered/{metric}_{version}.csv')
			.sort_values(['segmentation_id', 'patient_id'])
				.groupby('patient_id')
				.mean_test_score.apply(list).reset_index()
				.rename({'mean_test_score': 'unregistered'}, axis='columns')
		)
		.assign(statistic=lambda y: y.apply(wilcoxon_score, axis='columns'))
		.assign(
			pvalue=lambda y: [x[1] if x else None for x in y.statistic],
			statistic=lambda y: [x[0] if x else None for x in y.statistic],
		)
	[['patient_id', 'statistic', 'pvalue']]
)

t_test.to_csv(f'./show/{metric}-{version}-wilcoxon.csv', index=False, float_format='%.25f')
t_test

 C:\Users\Aleksas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\morestats.py:3141: UserWarning:Exact p-value calculation does not work if there are ties. Switching to normal approximation.
 C:\Users\Aleksas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\scipy\stats\morestats.py:3155: UserWarning:Sample size too small for normal approximation.


,patient_id,statistic,pvalue
0,2,3.5,1.024704e-01
1,15,60.0,6.757208e-01
2,28,23.0,1.992823e-01
3,32,3.0,2.980232e-07
4,35,74.0,3.979196e-01
5,39,39.0,4.296899e-04
6,40,NaN,NaN
7,41,5.0,2.341943e-01
8,45,46.0,1.027107e-03
9,50,159.0,9.368029e-01


In [32]:

metric = 'learning_rate'
# metric = 'reg_alpha'
version = 'both'

stats.wilcoxon(
	pd.read_csv(f'./roc/{metric}_{version}.csv')
		.sort_values(['segmentation_id', 'patient_id'])
		.mean_test_score
		.to_numpy(),
	pd.read_csv(f'./roc_unregistered/{metric}_{version}.csv')
		.sort_values(['segmentation_id', 'patient_id'])
		.mean_test_score
		.to_numpy(),
)

WilcoxonResult(statistic=8295.5, pvalue=5.622979822842875e-13)

In [ ]:

for segmentation_id in range(0, 25):
	plt.close()

	metric = 'learning_rate'
	if not os.path.exists(f'./roc/{metric}'):
		  os.makedirs(f'./roc/{metric}')
	grid_search = pd.read_csv(f'./roc/{metric}.csv')
	sns.scatterplot(
		data=grid_search.query(f'segmentation_id == {segmentation_id}'),
		y='mean_test_score', x=metric, hue='patient_id',
		palette=sns.color_palette('tab20')[:13],
	)
	plt.savefig(f'./roc/{metric}/segmentation-{segmentation_id}.png')

for segmentation_id in range(0, 25):
	plt.close()

	metric = 'reg_alpha'
	if not os.path.exists(f'./roc/{metric}'):
		  os.makedirs(f'./roc/{metric}')
	grid_search = pd.read_csv(f'./roc/{metric}.csv')
	sns.scatterplot(
		data=grid_search.query(f'segmentation_id == {segmentation_id}'),
		y='mean_test_score', x=metric, hue='patient_id',
		palette=sns.color_palette('tab20')[:13],
	)
	plt.savefig(f'./roc/{metric}/segmentation-{segmentation_id}.png')

for segmentation_id in range(0, 25):
	plt.close()

	metric = 'reg_lambda'
	if not os.path.exists(f'./roc/{metric}'):
		  os.makedirs(f'./roc/{metric}')
	grid_search = pd.read_csv(f'./roc/{metric}.csv')
	sns.scatterplot(
		data=grid_search.query(f'segmentation_id == {segmentation_id}'),
		y='mean_test_score', x=metric, hue='patient_id',
		palette=sns.color_palette('tab20')[:13],
	)
	plt.savefig(f'./roc/{metric}/segmentation-{segmentation_id}.png')
plt.close()
